In [1]:
import glob
futureList = glob.glob('C:/my_file/0_research/20250908_FutureStrategy/future_data/processed_parquet/*.parquet')
print(futureList)

import pandas as pd

df_fut = pd.read_parquet(futureList[0])

for future_file in futureList[1:]:
    df_future = pd.read_parquet(future_file)
    
    df_fut = pd.concat([df_fut, df_future], axis=0, ignore_index=True)


# 檢查結果
# print(df_opt.info())
print(df_fut.columns)


df_opt = pd.read_parquet("C:/my_file/0_research/20250908_FutureStrategy/option_data/processed_parquet/opt_all.parquet")
df_opt = df_opt.drop(columns=['漲跌價', '漲跌%', 'Unnamed: 20', '契約到期日', 'Unnamed: 21'])
print(df_opt.columns)



['C:/my_file/0_research/20250908_FutureStrategy/future_data/processed_parquet\\2001_fut.parquet', 'C:/my_file/0_research/20250908_FutureStrategy/future_data/processed_parquet\\2002_fut.parquet', 'C:/my_file/0_research/20250908_FutureStrategy/future_data/processed_parquet\\2003_fut.parquet', 'C:/my_file/0_research/20250908_FutureStrategy/future_data/processed_parquet\\2004_fut.parquet', 'C:/my_file/0_research/20250908_FutureStrategy/future_data/processed_parquet\\2005_fut.parquet', 'C:/my_file/0_research/20250908_FutureStrategy/future_data/processed_parquet\\2006_fut.parquet', 'C:/my_file/0_research/20250908_FutureStrategy/future_data/processed_parquet\\2007_fut.parquet', 'C:/my_file/0_research/20250908_FutureStrategy/future_data/processed_parquet\\2008_fut.parquet', 'C:/my_file/0_research/20250908_FutureStrategy/future_data/processed_parquet\\2009_fut.parquet', 'C:/my_file/0_research/20250908_FutureStrategy/future_data/processed_parquet\\2010_fut.parquet', 'C:/my_file/0_research/202509

In [28]:
import pandas as pd
import numpy as np

def clean_numeric_col(series):
    """
    輔助函式：清洗含有逗號的數字字串，並處理 '-' 為 NaN
    """
    return pd.to_numeric(
        series.astype(str).str.replace(',', '').str.strip().replace('-', np.nan), 
        errors='coerce'
    )

def clean_futures_data(df_raw):
    """
    清洗期貨資料
    1. 轉換日期格式
    2. 排除價差單 (含有 '/' 的合約)
    3. 轉換價格欄位為浮點數
    """
    print("--- 開始清洗期貨資料 (Futures) ---")
    df = df_raw.copy()
    
    # 1. 日期標準化
    df['交易日期'] = pd.to_datetime(df['交易日期'])
    
    # 2. 排除價差單 (Spread Orders)
    # 檢查 '到期月份(週別)' 和 '契約' 是否含有 '/'
    mask_spread_month = df['到期月份(週別)'].astype(str).str.contains('/')
    mask_spread_contract = df['契約'].astype(str).str.contains('/')
    
    before_len = len(df)
    df = df[~(mask_spread_month | mask_spread_contract)]
    after_len = len(df)
    print(f">> 已排除價差單: {before_len - after_len} 筆")

    # 3. 數值欄位清洗 (去除逗號, 轉 float)
    target_cols = ['開盤價', '最高價', '最低價', '收盤價', '結算價']
    for col in target_cols:
        if col in df.columns:
            df[col] = clean_numeric_col(df[col])
            
    # 4. 排序與重設索引
    df.sort_values(by=['交易日期', '契約', '到期月份(週別)'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    print(f">> 期貨資料清洗完成，共 {len(df)} 筆。")
    return df

def clean_options_data(df_raw):
    """
    清洗選擇權資料
    1. 過濾非一般交易時段
    2. 轉換日期與履約價格式
    3. 轉換價格欄位
    """
    print("--- 開始清洗選擇權資料 (Options) ---")
    df = df_raw.copy()
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].astype(str).str.strip()
    
    # 1. 過濾交易時段 (只留 '一般')
    if '交易時段' in df.columns:
        before_len = len(df)
        df = df[df['交易時段'] == '一般']
        print(f">> 已過濾盤後資料: {before_len - len(df)} 筆")
    
    # 2. 日期標準化
    df['交易日期'] = pd.to_datetime(df['交易日期'])
    
    # 3. 履約價轉數值 (重要：用於排序與查找)
    df['履約價'] = clean_numeric_col(df['履約價'])
    
    # 4. 價格欄位清洗
    target_cols = ['開盤價', '最高價', '最低價', '收盤價', '結算價']
    for col in target_cols:
        if col in df.columns:
            df[col] = clean_numeric_col(df[col])
            
    # 5. 確保到期月份格式乾淨 (去除空格)
    df['到期月份(週別)'] = df['到期月份(週別)'].astype(str).str.strip()

    # 6. 排序
    # 注意：履約價排序對於尋找價差組合(Spread)很重要
    df.sort_values(by=['交易日期', '到期月份(週別)', '履約價'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    print(f">> 選擇權資料清洗完成，共 {len(df)} 筆。")
    return df

# ==========================================
# 執行清洗 (假設 df_fut 和 df_opt 已經讀入)
# ==========================================
# 為了避免變數名稱衝突，建議使用 _clean 後綴
df_fut_clean = clean_futures_data(df_fut)
df_opt_clean = clean_options_data(df_opt)

# 簡單檢查一下結果
print("\n[檢查] 期貨資料前 3 筆:")
print(df_fut_clean[['交易日期', '契約', '到期月份(週別)', '開盤價', '收盤價']].head(3))

print("\n[檢查] 選擇權資料前 3 筆:")
print(df_opt_clean[['交易日期', '到期月份(週別)', '履約價', '買賣權', '收盤價']].head(3))

--- 開始清洗期貨資料 (Futures) ---
>> 已排除價差單: 36798 筆
>> 期貨資料清洗完成，共 44990 筆。
--- 開始清洗選擇權資料 (Options) ---
>> 已過濾盤後資料: 2317932 筆
>> 選擇權資料清洗完成，共 1529628 筆。

[檢查] 期貨資料前 3 筆:
        交易日期  契約 到期月份(週別)     開盤價     收盤價
0 2001-01-02  TX   200101  4702.0  4892.0
1 2001-01-02  TX   200102  4750.0  4909.0
2 2001-01-02  TX   200103  4724.0  4950.0

[檢查] 選擇權資料前 3 筆:
        交易日期 到期月份(週別)     履約價 買賣權  收盤價
0 2017-01-03   201701  7500.0  買權  0.0
1 2017-01-03   201701  7500.0  賣權  0.1
2 2017-01-03   201701  7600.0  買權  0.0


In [11]:
df_fut_clean

,交易日期,契約,到期月份(週別),開盤價,最高價,最低價,收盤價,漲跌價,漲跌%,成交量,結算價,未沖銷契約數,最後最佳買價,最後最佳賣價,歷史最高價,歷史最低價,是否因訊息面暫停交易,交易時段,價差對單式委託成交量
0,2001-01-02,TX,200101,4702.0,4948.0,4685.0,4892.0,122,2.56%,14824,4892.0,6699,4892.0,4893.0,5612,4561,NaN,NaN,NaN
1,2001-01-02,TX,200102,4750.0,4948.0,4666.0,4909.0,174,3.67%,102,4909.0,129,4890.0,4910.0,4948,4570,NaN,NaN,NaN
2,2001-01-02,TX,200103,4724.0,4950.0,4724.0,4950.0,112,2.32%,4,4919.0,34,4808.0,5030.0,9500,4635,NaN,NaN,NaN
3,2001-01-02,TX,200106,5030.0,5030.0,4900.0,4901.0,153,3.22%,4,4901.0,21,4900.0,4977.0,8700,4680,NaN,NaN,NaN
4,2001-01-02,TX,200109,0.0,0.0,0.0,0.0,-,-,0,4988.0,0,0.0,0.0,4953,4780,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44985,2026-01-16,TX,202606,31470.0,31700.0,31462.0,31700.0,355,1.13%,45,0.0,0,31550.0,31641.0,31700,20819,None,盤後,None
44986,2026-01-16,TX,202609,31400.0,31560.0,31273.0,31514.0,370,1.19%,15,31527.0,200,31510.0,31544.0,31560,24962,None,一般,None
44987,2026-01-16,TX,202609,31294.0,31500.0,31260.0,31500.0,356,1.14%,14,0.0,0,31323.0,31436.0,31500,24962,None,盤後,None
44988,2026-01-16,TX,202612,31502.0,31670.0,31370.0,31615.0,353,1.13%,12,31634.0,66,31609.0,31659.0,31670,27382,None,一般,None


In [34]:
import pandas as pd
import numpy as np

def get_target_contract_price(df_fut_clean, date, contract_month):
    """
    取得特定日期、特定合約月份的期貨價格。
    
    參數:
        df_fut_clean (pd.DataFrame): 經過 clean_futures_data 清洗後的期貨資料表。
        date (pd.Timestamp or str): 查詢日期。
        contract_month (str): 目標合約月份 (e.g., '202205')。
        
    回傳:
        float: 該合約當日的價格 (優先順序: 開盤價 > 收盤價 > 結算價)。
        None: 若查無資料。
    """
    
    # 1. 確保日期格式一致
    target_date = pd.to_datetime(date)
    
    # 2. 確保合約月份格式一致 (去除空白, 轉字串)
    target_month = str(contract_month).strip()
    
    # 3. 篩選日期 (利用 Boolean Mask)
    # 註：若資料量極大，可考慮將交易日期設為 Index 以加速，但此處維持通用性
    day_data = df_fut_clean[df_fut_clean['交易日期'] == target_date].copy()
    
    if day_data.empty:
        print(f">> 查無 {target_date.date()} 的期貨資料。")
        return None
        
    # 4. 篩選合約月份
    day_data['到期月份(週別)'] = day_data['到期月份(週別)'].str.strip()
    contract_data = day_data[day_data['到期月份(週別)'] == target_month]
    del day_data  # 釋放記憶體
    
    if contract_data.empty:
        print(f">> 查無 {target_date.date()} 的 {target_month} 合約資料。")
        return None
        
    # 5. 取得第一筆資料 (理論上清洗後同一天同一合約只會有一筆)
    row = contract_data.iloc[0]
    
    # 6. 價格選取邏輯 (Open > Close > Settlement)
    price = np.nan
    
    # 嘗試取得開盤價
    if '開盤價' in row and pd.notna(row['開盤價']) and row['開盤價'] > 0:
        price = row['開盤價']
    # Fallback: 收盤價
    elif '收盤價' in row and pd.notna(row['收盤價']) and row['收盤價'] > 0:
        price = row['收盤價']
    # Fallback: 結算價 (處理無成交量但有結算價的情況)
    elif '結算價' in row and pd.notna(row['結算價']) and row['結算價'] > 0:
        price = row['結算價']
        
    # 7. 回傳結果
    if pd.isna(price) or price <= 0:
        print(f">> {target_date.date()} 的 {target_month} 合約無有效價格資料。")
        return None
    
    return float(price)

# ==========================================
# 測試範例
# ==========================================
# 假設 df_fut_clean 已經存在
target_date = pd.Timestamp('2020-11-17')
contract = '202101'
price = get_target_contract_price(df_fut_clean, target_date, contract)
print(f"日期: {target_date.date()}, 合約: {contract}, 價格: {price}")

日期: 2020-11-17, 合約: 202101, 價格: 13531.0


In [40]:
import pandas as pd
import numpy as np


from datetime import datetime, timedelta
def weekday_count(y,m,weekday="Wed",count=3):
    weekday_map = {"Mon":0, "Tue":1, "Wed":2, "Thu":3, "Fri":4, "Sat":5, "Sun":6}
    target_weekday = weekday_map.get(weekday, 2)  # Default to Wednesday if not found
    first_day = datetime(y, m, 1)
    weekday_count = 0
    temp = first_day
    while weekday_count < count:
        if temp.weekday() == target_weekday:
            weekday_count += 1
        if weekday_count < count:
            temp += timedelta(days=1)
    return temp
    
def get_expiry_date(contract_str):
    try:
        s = str(contract_str).split('.')[0].strip()

        y = int(s[:4])
        m = int(s[4:6])
        if len(s) > 6: 
            weekSymbol = s[6]
            count = int(s[7])
            if weekSymbol in ['W','w']:
                return weekday_count(y, m, "Wed", count)
            elif weekSymbol in ['F','f']:
                return weekday_count(y, m, "Fri", count)
            else:
                return pd.NaT
        return weekday_count(y, m, "Wed", 3)
    except:
        return pd.NaT
import numpy as np
import pandas as pd
import py_lets_be_rational as lj

def get_greeks(df_opt, nowDate, S, R):
    now_df = df_opt[df_opt['交易日期'] == nowDate]
    call_df = now_df[now_df['買賣權']=='買權']
    put_df  = now_df[now_df['買賣權']=='賣權']
    
    
    """
    1. 先計算 Implied Volatility (IV)
    2. 再使用 IV 計算 Greeks
    """
    # 避免 SettingWithCopyWarning
    call_df = call_df.copy()
    put_df = put_df.copy()

    # ==========================================
    # 1. 時間前處理 (T & dT)
    # ==========================================
    # 假設 get_expiry_date 函式已定義
    call_df['T'] = call_df['到期月份(週別)'].apply(get_expiry_date)
    put_df['T'] = put_df['到期月份(週別)'].apply(get_expiry_date)

    # 計算年化剩餘時間，並設定極小值避免除以零
    call_df['dT'] = ((call_df['T'] - call_df['交易日期']).dt.days / 365.0).clip(lower=1e-5)
    put_df['dT'] = ((put_df['T'] - put_df['交易日期']).dt.days / 365.0).clip(lower=1e-5)

    # ==========================================
    # 2. 定義 IV 計算函式
    # ==========================================
    def calculate_iv(row, option_type_flag):
        """
        option_type_flag: 1.0 for Call, -1.0 for Put
        """
        price = row['收盤價']
        K = row['履約價']
        T = row['dT']
        
        # 簡易檢查：價格異常或時間歸零直接回傳 0
        if price <= 0 or T <= 0:
            return 0.0

        # 計算遠期價格 F = S * exp(R*T)
        F = S * np.exp(R * T)

        # 檢查內含價值 (Intrinsic Value) 防止套利違規導致錯誤
        intrinsic = max(0, F - K) if option_type_flag == 1 else max(0, K - F)
        if price <= intrinsic:
            return 0.0  # 價格低於內含價值，理論上 IV 為 0 或無解

        try:
            # 使用 py_lets_be_rational 反推 IV
            # 參數順序: price, F, K, T, q
            iv = lj.implied_volatility_from_a_transformed_rational_guess(
                price, F, K, T, option_type_flag
            )
            return iv
        except:
            return 0.0

    # ==========================================
    # 3. 執行 IV 計算
    # ==========================================
    # Call: q = 1.0
    call_df['Implied_Volatility'] = call_df.apply(lambda row: calculate_iv(row, 1.0), axis=1)
    
    # Put: q = -1.0
    put_df['Implied_Volatility'] = put_df.apply(lambda row: calculate_iv(row, -1.0), axis=1)

    # ==========================================
    # 4. 定義 Greeks 計算函式 (Black-Scholes)
    # ==========================================
    def calculate_bs_greeks(row, option_type_str):
        K = row['履約價']
        T = row['dT']
        sigma = row['Implied_Volatility']
        
        if sigma <= 0 or T <= 0:
            return 0.0, 0.0, 0.0, 0.0, 0.0
            
        sqrt_T = np.sqrt(T)
        d1 = (np.log(S / K) + (R + 0.5 * sigma ** 2) * T) / (sigma * sqrt_T)
        d2 = d1 - sigma * sqrt_T

        # 使用 lj.norm_cdf
        nd1 = lj.norm_cdf(d1)
        nd2 = lj.norm_cdf(d2)
        n_prime_d1 = (1.0 / np.sqrt(2 * np.pi)) * np.exp(-0.5 * d1 ** 2)

        # Delta, Itm Probability
        if option_type_str == 'call':
            delta = nd1
            itm_prob = nd2
        else:
            delta = nd1 - 1.0
            itm_prob = 1.0 - nd2
        # Gamma
        gamma = n_prime_d1 / (S * sigma * sqrt_T)

        # Vega
        vega = S * sqrt_T * n_prime_d1

        # Theta
        theta_common = -(S * sigma * n_prime_d1) / (2 * sqrt_T)
        if option_type_str == 'call':
            theta = theta_common - R * K * np.exp(-R * T) * nd2
        else:
            n_neg_d2 = 1.0 - nd2
            theta = theta_common + R * K * np.exp(-R * T) * n_neg_d2
            
        return delta, gamma, theta, vega, itm_prob

    # ==========================================
    # 5. 執行 Greeks 計算
    # ==========================================
    # Call Greeks
    greeks_call = call_df.apply(lambda row: calculate_bs_greeks(row, 'call'), axis=1, result_type='expand')
    call_df[['Delta', 'Gamma', 'Theta', 'Vega', 'Itm_Prob']] = greeks_call

    # Put Greeks
    greeks_put = put_df.apply(lambda row: calculate_bs_greeks(row, 'put'), axis=1, result_type='expand')
    put_df[['Delta', 'Gamma', 'Theta', 'Vega', 'Itm_Prob']] = greeks_put
    
    return call_df, put_df



def market_data_generator(start_date, end_date, df_opt, df_fut, risk_free_rate=0.01):
    """
    逐日生成市場資料生成器 (Generator)
    
    Yields:
        tuple: (current_date, S, call_df, put_df)
        
        - current_date (pd.Timestamp): 當前交易日
        - S (float): 當日標的價格 (使用近月期貨價格)
        - call_df (pd.DataFrame): 當日 Call 資料表 (含 Greeks)
        - put_df (pd.DataFrame): 當日 Put 資料表 (含 Greeks)
    """
    
    print(f"--- 初始化市場資料生成器 ({start_date} to {end_date}) ---")
    
    # 1. 建立交易日曆 (只取期貨有資料的日子，並限制在回測區間內)
    # 確保 index 重設，方便後續操作
    all_dates = df_fut['交易日期'].unique()
    # all_dates = pd.to_datetime(all_dates).sort_values()
    all_dates = all_dates[pd.to_datetime(all_dates).argsort()]
    # all_dates = pd.DataFrame(all_dates).sort_values(by=0)[0]
    
    mask_date = (all_dates >= pd.to_datetime(start_date)) & (all_dates <= pd.to_datetime(end_date))
    trade_dates = all_dates[mask_date]
    
    print(f">> 預計執行交易日數: {len(trade_dates)} 天")

    # 2. 逐日迴圈
    for current_date in trade_dates:
        
        # ==========================================
        # A. 取得當日標的價格 S (Near Month Future)
        # ==========================================
        # 篩選當日期貨資料
        daily_fut = df_fut[df_fut['交易日期'] == current_date]
        
        if daily_fut.empty:
            continue
            
        # 找出「近月合約」：排序「到期月份」，取第一筆
        # 假設資料已清洗過，無價差單，且格式正確
        daily_fut_sorted = daily_fut.sort_values(by='到期月份(週別)')
        
        # 取最近月合約
        near_month_row = daily_fut_sorted.iloc[0]
        
        # 決定價格 (Open > Close > Settlement)
        S = near_month_row['開盤價']
        if pd.isna(S) or S <= 0:
            S = near_month_row['收盤價']
        if pd.isna(S) or S <= 0:
            S = near_month_row['結算價']
            
        # 若仍無價格，跳過該日
        if pd.isna(S) or S <= 0:
            # print(f"Warning: {current_date.date()} 查無有效標的價格 S，跳過。")
            continue

        # ==========================================
        # B. 準備當日選擇權資料
        # ==========================================
        # 為了效能，先從大表切出當日資料
        daily_opt = df_opt[df_opt['交易日期'] == current_date]
        
        if daily_opt.empty:
            continue

        # ==========================================
        # C. 計算 Greeks
        # ==========================================
        # 呼叫您提供的 get_greeks 函式
        # 注意：get_greeks 會回傳 (call_df, put_df)
        try:
            # 這裡傳入 daily_opt，get_greeks 內部會再 filter 一次 date，這沒問題
            call_greeks, put_greeks = get_greeks(daily_opt, current_date, S, risk_free_rate)
            
            # 簡單防呆：確保回傳不是空的
            if call_greeks.empty and put_greeks.empty:
                continue
                
            # ==========================================
            # D. Yield 結果
            # ==========================================
            yield current_date, S, call_greeks, put_greeks
            
        except Exception as e:
            print(f"Error on {current_date.date()}: {e}")
            continue

print(">> Generator 函式定義完成。")

>> Generator 函式定義完成。


In [41]:
# ==========================================
# Generator 測試腳本
# ==========================================

# 1. 設定測試區間 (跑個幾天就好)
TEST_START = '2022-01-01'  # 請確保您的資料包含這段時間
TEST_END   = '2022-01-15'

print(f"--- 開始測試 Generator: {TEST_START} ~ {TEST_END} ---\n")

# 2. 初始化 Generator
# 注意：這裡傳入的是已經清洗過的 df_opt_clean 與 df_fut_clean
gen = market_data_generator(
    start_date=TEST_START, 
    end_date=TEST_END, 
    df_opt=df_opt_clean, 
    df_fut=df_fut_clean, 
    risk_free_rate=0.01
)

# 3. 執行並觀察輸出
try:
    count = 0
    for date, S, calls, puts in gen:
        count += 1
        print(f"[{count}] 日期: {date.date()} | 標的價格 S: {S}")
        print(f"    > Call 數量: {len(calls)}, Put 數量: {len(puts)}")
        
        # 檢查 Greeks 是否計算成功 (檢查 Delta 是否存在且不全為 0)
        # 隨機抽樣看一筆價平附近的資料 (假設 S=18000, 找 18000 附近的)
        if not calls.empty:
            # 找最接近 S 的履約價
            nearest_idx = (calls['履約價'] - S).abs().idxmin()
            sample_call = calls.loc[nearest_idx]
            
            print(f"    > [檢查] 履約價 {sample_call['履約價']} Call:")
            print(f"      IV    : {sample_call['Implied_Volatility']:.4f}")
            print(f"      Delta : {sample_call['Delta']:.4f}")
            print(f"      Vega  : {sample_call['Vega']:.4f}")
            
            if sample_call['Implied_Volatility'] == 0:
                print("      [警示] IV 為 0，可能是資料異常或該履約價無成交。")
        
        print("-" * 30)
        
        # 只測前 3 天就停，節省時間
        if count >= 3:
            print(">> 測試結束：前 3 天資料讀取成功。")
            break
            
except Exception as e:
    print(f"\n[錯誤] Generator 執行失敗: {e}")
    import traceback
    traceback.print_exc()

--- 開始測試 Generator: 2022-01-01 ~ 2022-01-15 ---

--- 初始化市場資料生成器 (2022-01-01 to 2022-01-15) ---
>> 預計執行交易日數: 10 天
[1] 日期: 2022-01-03 | 標的價格 S: 18288.0
    > Call 數量: 344, Put 數量: 344
    > [檢查] 履約價 18300.0 Call:
      IV    : 0.1243
      Delta : 0.5019
      Vega  : 1527.5138
------------------------------
[2] 日期: 2022-01-04 | 標的價格 S: 18361.0
    > Call 數量: 347, Put 數量: 347
    > [檢查] 履約價 18350.0 Call:
      IV    : 0.4545
      Delta : 0.5153
      Vega  : 383.1267
------------------------------
[3] 日期: 2022-01-05 | 標的價格 S: 18480.0
    > Call 數量: 414, Put 數量: 414
    > [檢查] 履約價 18500.0 Call:
      IV    : 0.1262
      Delta : 0.4937
      Vega  : 1443.6931
------------------------------
>> 測試結束：前 3 天資料讀取成功。
